In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from lib.m import *
from lib.eda_visualization import *

In [19]:
df = pd.read_csv("df_complete.csv").iloc[:,3:]
df

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,company_name,company_about,founded,business model,employees,funding stage,raised,product stage,status,geographical markets,...,tag_e-mobility,targetmarket_public-transportation,targetmarket_smart-mobility,tag_hydraulic-drive,tag_hud,tag_simulation-software,tag_luggage,tag_traffic-violations,tag_car-audio,tag_trip
0,Tastewise,Tastewise is an AI platform designed to help f...,7/2017,B2B,51-200,ROUND A,$21.5M,Released,active,"australia, canada, france, india, united kingd...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wilk Technologies,Wilk is dedicated to revolutionizing the dairy...,6/2018,"B2B, B2B2C",11-50,Public,$4.69M,R&D,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eco Pack Green Box,Eco Pack Green Box has developed and patented ...,3/2008,B2B,11-50,Revenue Financed,NaN,Released,not_active,"canada, mexico, spain, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BeeHero,BeeHero has developed a platform that can pred...,10/2017,B2B,1-10,ROUND A,$24M,Released,active,"global, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cham Foods,Cham Foods is a multinational company with man...,12/1970,"B2B, B2B2C",11-50,Public,NaN,Released,active,"north america, europe, global, france, germany...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13043,TriEye,TriEye is a fabless semiconductor company that...,11/2017,B2B,51-200,ROUND A,$96M,R&D,active,global,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044,LYNX Smartcars,LYNX is developing software for connected and ...,1/2016,B2B,1-10,Bootstrapped,NaN,R&D,not_active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13045,Deeyook Location Technologies,Deeyook seeks to redefine location technology ...,3/2017,B2B,11-50,Seed,NaN,Released,active,global,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13046,SafeCue,SafeCue combines the power of deep learning wi...,1/2016,B2B,1-10,Seed,$500K,Beta,not_active,"asia, germany, india, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_test = df.copy()
df_test.shape

(13048, 2868)

In [21]:
tag_cols = [col for col in df_test.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_test.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_test.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_test.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_test.columns if col.startswith("core_technology_")]


In [22]:
df_test[tag_cols] = df_test[tag_cols].fillna(0)
df_test[targetmarket_cols] = df_test[targetmarket_cols].fillna(0)
df_test[sector_list] = df_test[sector_list].fillna(0)
df_test[target_ind_list] = df_test[target_ind_list].fillna(0)
df_test[technology_list] = df_test[technology_list].fillna(0)


vectorize

In [7]:
# df_test = vectorize_and_replace(df_test, tag_cols, "tag")
# df_test = vectorize_and_replace(df_test, targetmarket_cols, "targetmarket")
# df_test = vectorize_and_replace(df_test, sector_list, "sector")
# df_test = vectorize_and_replace(df_test, target_ind_list, "target_ind")
# df_test = vectorize_and_replace(df_test, technology_list, "tech")
# df_test.shape

Replacing null values to 0

In [23]:
df_test['raised'] = df_test['raised'].fillna(0)
df_test['total_rounds'] = df_test['total_rounds'].fillna(0)
df_test['investors'] = df_test['investors'].fillna(0)
df_test['ipo_price'] = df_test['ipo_price'].fillna(0)


In [24]:
df_test.status.replace({'active' : 1, 'not_active' : 0 }, inplace=True)

Removing unused columns

In [25]:
df_test.drop(['use cases','academic spin-off','total_raised'], axis = 1, inplace = True)
df_test.shape

(13048, 2865)

Decoding str to numeric value

In [26]:
df_test['ipo_price'] = conv_to_float(df_test["ipo_price"])
df_test['raised'] = conv_to_float(df_test["raised"])

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
df_test

,company_about,founded,business model,employees,funding stage,raised,product stage,status,geographical markets,products,...,tag_application-optimization,tag_mainframe,tag_sourcing,tag_plastic-surgery,tag_dermatology,tag_card-payments,tag_paycheck,tag_remittances,targetmarket_unbanked,tag_it-architecture
0,Tastewise is an AI platform designed to help f...,7/2017,B2B,51-200,ROUND A,21500000,Released,1,"australia, canada, france, india, united kingd...",tastewise,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Wilk is dedicated to revolutionizing the dairy...,6/2018,"B2B, B2B2C",11-50,Public,4690000,R&D,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Eco Pack Green Box has developed and patented ...,3/2008,B2B,11-50,Revenue Financed,0,Released,0,"canada, mexico, spain, united states",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BeeHero has developed a platform that can pred...,10/2017,B2B,1-10,ROUND A,24000000,Released,1,"global, united states",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cham Foods is a multinational company with man...,12/1970,"B2B, B2B2C",11-50,Public,0,Released,1,"north america, europe, global, france, germany...",NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4983,"Zaponet is analytics solutions provider, offer...",2011,B2B,11-50,Bootstrapped,0,Released,0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4984,Storydoc can easily transform static PDFs and ...,1/2020,B2B,1-10,Bootstrapped,0,Released,1,global,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4985,PatX developed an AI system for the assessment...,8/2019,B2B,1-10,Bootstrapped,0,Alpha,1,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4986,Peck's SaaS platform enables successful digita...,8/2019,B2B,1-10,Bootstrapped,0,Beta,1,north america,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Converting the numeric colums

In [27]:

df_test['business model']= preprocessing.LabelEncoder().fit_transform(df_test['business model'])

In [28]:
df_test['raised'] = df_test['raised'].astype('float')
df_test['total_rounds'] = df_test['total_rounds'].astype('int')
df_test['investors'] = df_test['investors'].astype('int')
df_test['ipo_price'] = df_test['ipo_price'].astype('float')
df_test['business model'] = df_test['business model'].astype('int')

In [30]:
df_test.ipo_price.mean()

841852.5444512569

,business model,raised,status,total_rounds,investors,ipo_price,sector_AgriFood-tech & Water,target_industry_Food Retail & Consumption,target_industry_Food Retail & Consumption_Home,target_industry_Food Retail & Consumption_Restaurants,...,tag_application-optimization,tag_mainframe,tag_sourcing,tag_plastic-surgery,tag_dermatology,tag_card-payments,tag_paycheck,tag_remittances,targetmarket_unbanked,tag_it-architecture
business model,1.000000,-0.074760,-0.211418,-0.111628,-0.120556,-0.020432,-0.135069,-0.002908,0.047218,-0.025575,...,0.009444,-0.012474,0.016750,-0.012474,-0.012474,-0.012474,-0.012474,-0.012474,-0.012474,-0.012474
raised,-0.074760,1.000000,0.107865,0.484236,0.539615,0.273082,-0.021269,0.002027,-0.008558,-0.000600,...,-0.002526,-0.002701,-0.002504,-0.002482,-0.002482,-0.002569,-0.002569,-0.002569,-0.002569,0.010522
status,-0.211418,0.107865,1.000000,0.147338,0.149546,0.037352,0.195103,0.024337,0.028492,-0.008091,...,0.011364,0.011364,0.011364,0.011364,0.011364,-0.017645,-0.017645,-0.017645,-0.017645,0.011364
total_rounds,-0.111628,0.484236,0.147338,1.000000,0.756064,0.129430,0.007692,0.022210,-0.003494,0.011155,...,0.002199,-0.008336,0.002199,0.002199,0.002199,0.002199,0.002199,0.002199,0.002199,0.023269
investors,-0.120556,0.539615,0.149546,0.756064,1.000000,0.118450,-0.023250,0.011674,-0.003643,0.007199,...,0.004991,-0.006455,-0.000732,-0.000732,-0.000732,-0.000732,-0.000732,-0.000732,-0.000732,0.027884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tag_card-payments,-0.012474,-0.002569,-0.017645,0.002199,-0.000732,-0.000707,-0.006486,-0.002502,-0.001439,-0.001785,...,-0.000201,-0.000201,-0.000201,-0.000201,-0.000201,1.000000,1.000000,1.000000,1.000000,-0.000201
tag_paycheck,-0.012474,-0.002569,-0.017645,0.002199,-0.000732,-0.000707,-0.006486,-0.002502,-0.001439,-0.001785,...,-0.000201,-0.000201,-0.000201,-0.000201,-0.000201,1.000000,1.000000,1.000000,1.000000,-0.000201
tag_remittances,-0.012474,-0.002569,-0.017645,0.002199,-0.000732,-0.000707,-0.006486,-0.002502,-0.001439,-0.001785,...,-0.000201,-0.000201,-0.000201,-0.000201,-0.000201,1.000000,1.000000,1.000000,1.000000,-0.000201
targetmarket_unbanked,-0.012474,-0.002569,-0.017645,0.002199,-0.000732,-0.000707,-0.006486,-0.002502,-0.001439,-0.001785,...,-0.000201,-0.000201,-0.000201,-0.000201,-0.000201,1.000000,1.000000,1.000000,1.000000,-0.000201


In [31]:
df_test.loc[(df_test["status"]==1)&(df_test['raised']>4000000), 'suceeded'] = 1
df_test.loc[(df_test["status"]==0)|(df_test['raised']<=4000000), 'suceeded'] = 0

In [33]:
df_test.to_csv('cleaned_complete.csv')

In [32]:
# df_test.loc["succeeded"] = df_test[(df_test['status'] == 1) & (df_test['raised'] > 5000000)]
df_test.shape

(13048, 2866)

In [127]:

xtrain, xtest, ytrain, ytest = train_test_split(df_test.iloc[:,:-1].select_dtypes(include=np.number), df_test.iloc[:,-1])


In [128]:
df_test.iloc[2930,:]

company_name                                                    SolarEdge
company_about           SolarEdge Technologies invented an inverter so...
founded                                                            8/2006
business model                                                          2
employees                                                            500+
funding stage                                                      Public
raised                                                        124700000.0
product stage                                                    Released
status                                                                  1
geographical markets    europe, asia, africa, global, australia, cambo...
products                                                              NaN
fund_stage                                                         Public
total_rounds                                                            6
investors                             

In [75]:
# TODO maybe a function for each vec col?
#      or maybe not vectorize !?

lst_vec = df_test.iloc[:,-6].tolist()
df_vec = pd.DataFrame(lst_vec)
df_vec.shape
xtrain, xtest, ytrain, ytest = train_test_split(df_vec, df_test.iloc[:,-1])

In [129]:
lr = LogisticRegression()
lr.fit(xtrain,ytrain)

LogisticRegression()

In [130]:
ytrain_pred = lr.predict(xtrain)
ytest_pred = lr.predict(xtest)

In [67]:
metrics.confusion_matrix(ytrain_pred, ytrain)

array([[3336,  208],
       [   7,  190]], dtype=int64)

In [131]:
print("Train results:")
print("accuracy is:",metrics.accuracy_score(ytrain_pred, ytrain))
print("precision is:",metrics.precision_score(ytrain_pred, ytrain))
print("recall is:",metrics.recall_score(ytrain_pred, ytrain))
print("f1 is:",metrics.f1_score(ytrain_pred, ytrain))
print("---------------------")
print("Test results:")
print("accuracy is:",metrics.accuracy_score(ytest_pred, ytest))
print("precision is:",metrics.precision_score(ytest_pred, ytest))
print("recall is:",metrics.recall_score(ytest_pred, ytest))
print("f1 is:",metrics.f1_score(ytest_pred, ytest))

Train results:
accuracy is: 0.7883711424483957
precision is: 1.0
recall is: 0.4123155505107832
f1 is: 0.5838858750251156
------------
Test results:
accuracy is: 0.7805027590435316
precision is: 1.0
recall is: 0.3822260569456428
f1 is: 0.5530586766541823


<strong>This is the results with vectorize</strong>